In [136]:
import pandas as pd
import numpy as np

In [137]:
data = pd.read_csv('new_data/complete-data.csv')

data.sort_values('year')

,playerID,year,tmID,GP,oRebounds,dRebounds,dq,PostPoints,ft%,fg%,...,career_year,playoff_progression_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,stats_post_rolling,PostPoints_rolling,W%_rolling,postW%_rolling,rpg_rolling
0,434,4,555,34.0,1.18,2.88,0.06,17.33,0.88,0.46,...,4,0.000000,-354.740000,0.000000,35.033333,0.000000,0.000000,0.388333,0.000000,3.230000
59,372,4,561,34.0,0.85,2.65,0.03,4.17,0.58,0.30,...,4,1.000000,-175.570000,0.000000,57.800000,-0.100000,4.733333,0.566667,0.100000,3.653333
58,399,4,566,33.0,0.12,1.15,0.00,4.00,0.56,0.28,...,4,1.666667,-94.730000,0.000000,22.300000,-0.080000,0.666667,0.613333,0.443333,1.583333
57,15,4,558,34.0,0.71,3.29,0.00,12.33,0.84,0.47,...,4,2.000000,-256.733333,0.333333,43.666667,-0.153333,11.943333,0.726667,0.443333,3.970000
56,46,4,556,34.0,0.47,1.03,0.00,2.00,0.67,0.35,...,4,0.333333,-142.516667,0.333333,45.933333,0.036667,1.556667,0.500000,0.110000,3.553333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,43,11,563,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,9,1.000000,-263.713333,0.000000,35.266667,0.020000,15.833333,0.580000,0.220000,2.346667
631,342,11,574,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,4,2.333333,-69.080000,0.000000,17.566667,-0.630000,0.000000,0.775000,0.460000,3.040000
630,40,11,564,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,11,2.000000,-141.776667,0.333333,38.233333,-0.653333,6.433333,0.590000,0.476667,2.266667
646,466,11,573,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,9,2.666667,-160.953333,0.000000,50.800000,-0.756667,11.200000,0.630000,0.473333,4.790000


In [138]:
with open('stats.txt', 'w') as file:
    for col in data.columns:
        if type(data[col][0]) == str: 
            continue
        file.write("-"*50 + "\n")
        file.write(f"Statistics of {col}\n")
        file.write("-"*50 + "\n")
        # data[col].plot.box();  # You can enable this if you want to create a plot

        q3, q1 = np.percentile(data[col], [75 ,25])

        file.write(f"Min: {data[col].min()}\n")
        file.write(f"Max: {data[col].max()}\n")
        file.write(f"Mean: {data[col].mean()}\n")
        file.write(f"Median: {data[col].median()}\n")
        file.write(f"Standard deviation: {data[col].std()}\n")
        file.write(f"Skewness: {data[col].skew()}\n")
        file.write(f"Variance: {data[col].var()}\n")
        file.write(f"Kurtosis: {data[col].kurtosis()}\n")
        file.write(f"Interquartile Range: {q3 - q1} | Q1: {q1} | Q3: {q3}\n")

In [139]:
outliers_std = {}
all_outliers = []  # Array to store all outlier rows

#columns to skip
not_skip_cols = ['height', 'weight']

for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    data_mean, data_std = data[col].mean(), data[col].std()
    # identify outliers
    cut_off = data_std * 4
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers = pd.concat([data.loc[data[col] < lower], data.loc[data[col] > upper]])
    outliers_std[col] = outliers
    all_outliers.extend(outliers.values.tolist())  # Add outliers to the array

for key in outliers_std.keys():
    if len(outliers_std[key]) > 0:
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_std[key][key])

# Printing all outlier rows
print("All outlier rows:")
for outlier in all_outliers:
    print(outlier)

All outlier rows:


In [140]:
outliers_ids_std = set([val for key in outliers_std.keys() for val in outliers_std[key]['playerID']])
outlier_ratio = len(outliers_ids_std) / len(data)
outlier_ratio

0.0

In [141]:
outliers_z = {}
for col in data.columns:
    if type(data[col][0]) == str or col not in not_skip_cols: 
        continue
    z_val = (data[col] - data[col].mean()) / data[col].std()
    outliers_z[col] = data.loc[(z_val < -4) | (z_val > 4)]

for key in outliers_z.keys():
    if (len(outliers_z[key]) > 0):
        print("-"*50)
        print(f"Outliers of {key} - mean: {data[key].mean()} - std: {data[key].std()}")
        print("-"*50)
        print(outliers_z[key][key])

In [142]:
outliers_ids_z = set([val for key in outliers_z.keys() for val in outliers_z[key]['playerID']])
outlier_ratio = len(outliers_ids_z) / len(data)
outlier_ratio

0.0

In [143]:
outliers_ids = outliers_ids_std.union(outliers_ids_z)
print(f"Total outliers: {len(outliers_ids)}")
print(f"Total outliers ratio: {len(outliers_ids) / len(data)}")
print(outliers_ids)
#create a new df with outliers removed
manual_outlier_df = data[~data['playerID'].isin(outliers_ids)]

manual_outlier_df.duplicated(subset=['playerID', 'year']).sum()

Total outliers: 0
Total outliers ratio: 0.0
set()


3

In [144]:

manual_outlier_df.to_csv('new_data/complete-data.csv', index=False)
manual_outlier_df.sort_values('year')
# manual_outlier_df.to_csv('output/loan_dev_transformed_without_outliers.csv', index=False)

,playerID,year,tmID,GP,oRebounds,dRebounds,dq,PostPoints,ft%,fg%,...,career_year,playoff_progression_rolling,eff_rolling,award_count_rolling,defensive_prowess_rolling,stats_post_rolling,PostPoints_rolling,W%_rolling,postW%_rolling,rpg_rolling
0,434,4,555,34.0,1.18,2.88,0.06,17.33,0.88,0.46,...,4,0.000000,-354.740000,0.000000,35.033333,0.000000,0.000000,0.388333,0.000000,3.230000
59,372,4,561,34.0,0.85,2.65,0.03,4.17,0.58,0.30,...,4,1.000000,-175.570000,0.000000,57.800000,-0.100000,4.733333,0.566667,0.100000,3.653333
58,399,4,566,33.0,0.12,1.15,0.00,4.00,0.56,0.28,...,4,1.666667,-94.730000,0.000000,22.300000,-0.080000,0.666667,0.613333,0.443333,1.583333
57,15,4,558,34.0,0.71,3.29,0.00,12.33,0.84,0.47,...,4,2.000000,-256.733333,0.333333,43.666667,-0.153333,11.943333,0.726667,0.443333,3.970000
56,46,4,556,34.0,0.47,1.03,0.00,2.00,0.67,0.35,...,4,0.333333,-142.516667,0.333333,45.933333,0.036667,1.556667,0.500000,0.110000,3.553333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,43,11,563,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,9,1.000000,-263.713333,0.000000,35.266667,0.020000,15.833333,0.580000,0.220000,2.346667
631,342,11,574,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,4,2.333333,-69.080000,0.000000,17.566667,-0.630000,0.000000,0.775000,0.460000,3.040000
630,40,11,564,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,11,2.000000,-141.776667,0.333333,38.233333,-0.653333,6.433333,0.590000,0.476667,2.266667
646,466,11,573,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,9,2.666667,-160.953333,0.000000,50.800000,-0.756667,11.200000,0.630000,0.473333,4.790000
